In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import multiprocessing
from multiprocessing import Pool
import os

In [10]:
to = 13
savepath = "graphs/test_only_advection_all"
file = "/home/Aaron/Isca_advection/IdealizeSpetral.jl/exp/HSt42/outputs/0523/test_only_advection_all.dat"


ds = h5py.File(file, "r")

u      = np.asarray(ds["grid_u_c_xyzt"])
v      = np.asarray(ds["grid_v_c_xyzt"])
spe_vor_final_xyz = np.asarray(ds["spe_vor_c_xyzt"])
grid_t_c_xyzt = np.asarray(ds["grid_t_c_xyzt"])
grid_t_eq_xyzt = np.asarray(ds["grid_t_eq_xyzt"])
grid_p_full_xyzt   = np.asarray(ds["grid_p_full_xyzt"])
qv      = np.asarray(ds["grid_tracers_c_xyz1t"])
grid_t_c_xyzt = np.asarray(ds["grid_t_c_xyzt"])


k = 0.5*(v**2+ u**2)
print(k.shape)
x = np.linspace(-180,180,128)
y = np.linspace(-90, 90,64)
xx, yy = np.meshgrid(x, y)

try:
    os.makedirs(f"{savepath}")
except FileExistsError:
    pass

def plot_all(t):
    fig, ax = plt.subplots(2, 2, figsize=(12, 12))
    fig.suptitle(f"Day {t}")
    cs0 = ax[0, 0].contourf(xx, yy, k[t, 19], levels=np.linspace(0, 800, 11), extend='max')
    fig.colorbar(cs0, orientation='horizontal')
    ax[0, 0].set_title(f"Kinetic Energy at surface")

    # cs1 = ax[0, 1].contourf(xx, yy, k[t, 0], levels=np.linspace(0, 800, 11), extend='max')
    # fig.colorbar(cs1, orientation='horizontal')
    # ax[0, 1].set_title(f"Kinetic Energy at top")
    # fig.colorbar(cs1, orientation='horizontal')
    # ax[0, 1].set_title(f"Kinetic Energy at top")

    cs1 = ax[0, 1].quiver(xx[::4, ::4], yy[::4, ::4], u[t, 19][::4, ::4], v[t, 19][::4, ::4], scale=500)
    ax[0, 1].quiverkey(cs1, 1.05, 1.05, 30, "30 m/s")
    ax[0, 1].set_title(f"Wind at surface")


    cs2 = ax[1, 0].contourf(xx, yy, qv[t, 19], levels=np.linspace(-100, 800, 11), extend='both')
    fig.colorbar(cs2, orientation='horizontal')
    ax[1, 0].set_title(f"qv at sfc")

    cs3 = ax[1, 1].contourf(xx, yy, grid_t_c_xyzt[t, 0], levels=np.linspace(250, 320, 11), extend='both')
    fig.colorbar(cs3, orientation='horizontal')
    ax[1, 1].set_title(f"T at sfc")

    for i in range(2):
        for j in range(2):
            ax[i, j].set_xlabel("Longitude")
            ax[i, j].set_ylabel("Latitude")

    plt.savefig(f"{savepath}/{t}.png", dpi=300.)
    plt.close()

(13, 20, 64, 128)


In [11]:
leap = 1
nProc = int(multiprocessing.cpu_count() // 2)
with Pool(nProc) as p:
    results = [p.apply_async(plot_all, (t, )) for t in range(0, to, leap)]
    final = [result.get() for result in results]
!cd $savepath && ffmpeg -r 5 -i %d.png -pix_fmt yuv420p 0.mov -y

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/tmp/build/80754af9/ffmpeg_1587154242452/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeho --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
